In [1]:
import numpy as np
import cv2
import os
from colorize.colorize import doColorize
from tqdm.notebook import tqdm

In [2]:
sequences_rootdir = '../SemanticKITTI/dataset/sequences'
sequences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

In [3]:
# Read the label data from files
@staticmethod
def readLabel(path):
    label = np.fromfile(path, dtype=np.int32)
    sem_label = label & 0xFFFF  # semantic label in lower half
    inst_label = label >> 16  # instance id in upper half
    return sem_label, inst_label

In [4]:
for seq in tqdm(sequences, desc="Processing Sequences"):
    basedir = os.path.join(sequences_rootdir, seq)
    pointdir = f'{basedir}/velodyne'
    labeldir = f'{basedir}/labels'
    imgdir = f'{basedir}/image_2'
    calib_file = f'{basedir}/calib.txt'
    # search all .bin files in pointdir and put them in a list
    point_files = sorted([f for f in os.listdir(pointdir) if f.endswith('.bin')])
    # search all .png files in imgdir and put them in a list
    img_files = sorted([f for f in os.listdir(imgdir) if f.endswith('.png')])
    # search all .label files in labeldir and put them in a list
    label_files = sorted([f for f in os.listdir(labeldir) if f.endswith('.label')])
    # make sure the number of point files and image files match
    if len(point_files) != len(img_files):
        raise ValueError("Number of point files and image files do not match.")
    if len(point_files) != len(label_files):
        raise ValueError("Number of point files and label files do not match.")

    # save the preprocess point cloud to a new file in the subfolder 'preprocess', use the same filename
    preprocess_dir = os.path.join(basedir, 'preprocess')
    os.makedirs(preprocess_dir, exist_ok=True)

    # go through the list of point files and image files and colorize the points
    for point_file, img_file, label_file in tqdm(zip(point_files, img_files, label_files), total=len(point_files), desc="Colorizing point clouds", leave=False):
        pc_file = os.path.join(pointdir, point_file)
        im_file = os.path.join(imgdir, img_file)
        label_file = os.path.join(labeldir, label_file)
        label,_ = readLabel(label_file)  
        points, pts_valid, colors = doColorize(pc_file, im_file, calib_file, save_only_valid_points=False, save_file=False)
        # check if length of points and labels match
        if len(points) != len(label):
            raise ValueError(f"Number of points ({len(points)}) does not match number of labels ({len(label)}) in file {label_file}.")
        # cascade the labels to the points
        points = np.hstack((points, label[:, np.newaxis]))  # add labels
        # save the points to a new file in the preprocess directory
        output_file = os.path.join(preprocess_dir, point_file)
        points.astype(np.float32).tofile(output_file)

Processing Sequences:   0%|          | 0/11 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4541 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4661 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/801 [00:00<?, ?it/s]

...

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1101 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/4071 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1591 [00:00<?, ?it/s]

Colorizing point clouds:   0%|          | 0/1201 [00:00<?, ?it/s]